# Know Your Fan - Protótipo para Fãs da FURIA 🐯🔥

Este é o notebook correspondente ao projeto **Know Your Fan**, criado para o **Challenge #2 da FURIA**. O objetivo é conhecer melhor os fãs do time e oferecer experiências personalizadas. Este notebook contém o mesmo código do `app.py`, mas em formato interativo para facilitar testes e ajustes.

### Funcionalidades
-  **Coletar dados básicos**: Nome, CPF, endereço, e-mails, jogos e jogadores favoritos, eventos e compras.
-  **Validar documentos**: Usa o Tesseract OCR para validar documentos como RG ou CPF.
-  **Conectar redes sociais**: Simula a integração com Twitter/X (devido ao plano gratuito).
-  **Recomendar experiências**: Sugere eventos, produtos e conteúdos personalizados com base no seu perfil, como os próximos campeonatos da FURIA, itens da loja oficial ou workshops.

## 1. Importações e Configurações 

Aqui importamos todas as bibliotecas necessárias para o projeto e configuramos o caminho do Tesseract (usado para validar documentos).

In [ ]:
import streamlit as st
import pandas as pd
import sqlite3
import os
import numpy as np
from sklearn.neighbors import NearestNeighbors
from uuid import uuid4
import tempfile
import pytesseract
from PIL import Image

if os.name == 'nt':
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
else:
    pytesseract.pytesseract.tesseract_cmd = 'tesseract'

## 2. Inicializar o Estado da Sessão e Banco de Dados 

Configuramos o estado da sessão para armazenar os dados do usuário e criamos um banco de dados SQLite para salvar as informações.

In [ ]:
# Inicializar o estado da sessão para armazenar dados do usuário e jogadores selecionados
if 'dados_usuario' not in st.session_state:
    st.session_state['dados_usuario'] = None
if 'jogadores_selecionados' not in st.session_state:
    st.session_state['jogadores_selecionados'] = []

# Conexão com SQLite
conn = sqlite3.connect('fan_data.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS usuarios (
    ID TEXT PRIMARY KEY,
    NOME TEXT,
    CPF TEXT,
    ENDERECO TEXT,
    EMAIL TEXT,
    JOGADORES TEXT,
    JOGOS TEXT,
    EVENTOS TEXT,
    COMPRAS TEXT
)''')
cursor.execute('''CREATE TABLE IF NOT EXISTS twitter_data (
    ID TEXT PRIMARY KEY,
    TWITTER_USERNAME TEXT,
    FOLLOWING TEXT,
    INTERACOES TEXT
)''')
conn.commit()
conn.close()

## 3. Configuração da Página e Estilo 

Configuramos o layout da página com o Streamlit e adicionamos um estilo personalizado com CSS, incluindo uma imagem de fundo da FURIA.

In [ ]:
# Configuração da página
st.set_page_config(page_title="Know Your Fan - FURIA", page_icon="https://cdn.dribbble.com/userupload/11627401/file/original-405f194bea083344029e99856c00f6f8.png?resize=1024x768&vertical=center", layout="wide")

st.markdown(
    """
    <style>
    /* Definir a imagem de fundo para o corpo da página */
    .stApp {
        background: url("https://pbs.twimg.com/media/F98rmOiWYAAFMyD?format=jpg&name=large");
        background-size: cover;
        background-position: center;
        background-attachment: fixed;
        color: white; /* Texto branco para contraste */
    }

    /* Adicionar um fundo preto opaco para os elementos do formulário */
    .stTextInput, .stSelectbox, .stTextArea, .stButton, .stForm {
        background-color: rgb(0, 0, 0); /* Fundo preto opaco */
        padding: 10px;
        border-radius: 5px;
    }

    /* Manter fundo semi-transparente para outras seções */
    .stMarkdown, .stFileUploader, .st.multiselect, .stSuccess, .stInfo {
        background-color: rgba(0, 0, 0, 0.7); /* Fundo preto semi-transparente */
        padding: 10px;
        border-radius: 5px;
    }

    /* Garantir que os cabeçalhos também sejam brancos */
    h1, h2, h3, h4, h5, h6 {
        color: white;
    }

    /* Ajustar a cor do texto dentro dos contêineres */
    .stMarkdown p, .stMarkdown div, .stTextInput input, .stTextArea textarea, .stSelectbox div, .stButton button {
        color: white;
    }

    /* Ajustar bordas dos contêineres para melhor contraste */
    .stContainer {
        border: 1px solid rgba(255, 255, 255, 0.2);
    }
    </style>
    """,
    unsafe_allow_html=True
)

## 4. Função para Coletar Dados do Usuário 

Esta função cria um formulário para coletar informações do usuário, como nome, CPF, endereço, jogos favoritos, e mais. Os dados são salvos em um banco SQLite.

In [ ]:
def coletar_dados_usuario():
    st.header("🔥 Queremos conhecer você!")
    st.write("Preencha os dados abaixo para que possamos personalizar sua experiência como FURIOSO.")
    with st.form("form_usuario"):
        nome = st.text_input("Nome completo")
        cpf = st.text_input("CPF")
        endereco = st.text_input("Endereço")
        email = st.text_input("E-mail")
        jogadores_favoritos = st.multiselect(
            "Jogadores preferidos (elenco atual de CS da FURIA)",
            ["FalleN", "yuurih", "KSCERATO", "chelo", "skullz", "guerri (coach)"]
        )
        jogos_favoritos = st.multiselect("Jogos favoritos", ["CS:GO", "Valorant", "League of Legends", "Outros"])
        eventos_ano = st.text_area("Quais eventos de esports que participou no último ano?")
        compras_ano = st.text_area("Quais produtos da FURIA você adquiriu recentemente?")
        submitted = st.form_submit_button("Enviar")
        if submitted:
            user_id = str(uuid4())
            conn = sqlite3.connect('fan_data.db')
            dados = pd.DataFrame({
                'ID': [user_id],
                'NOME': [nome],
                'CPF': [cpf],
                'ENDERECO': [endereco],
                'EMAIL': [email],
                'JOGADORES': [','.join(jogadores_favoritos)],
                'JOGOS': [','.join(jogos_favoritos)],
                'EVENTOS': [eventos_ano],
                'COMPRAS': [compras_ano]
            })
            dados.to_sql('usuarios', conn, if_exists='append', index=False)
            conn.close()
            st.session_state['dados_usuario'] = dados
            st.session_state['jogadores_selecionados'] = jogadores_favoritos
            st.success("Dados salvos com sucesso!")
    return st.session_state['dados_usuario'], st.session_state['jogadores_selecionados']

## 5. Função para Validar Documentos 

Esta função usa o Tesseract OCR para validar documentos (como RG ou CPF) a partir de imagens.

In [ ]:
# Função para validar documentos usando pytesseract
def validar_documento(imagem_path):
    try:
        image = Image.open(imagem_path)
        texto = pytesseract.image_to_string(image, lang='eng')
        if texto:
            digits = ''.join([c for c in texto if c.isdigit()])
            if len(digits) >= 11:
                return True, texto
            return False, "Documento inválido: número de CPF não encontrado"
        return False, "Nenhum texto detectado"
    except Exception as e:
        return False, f"Erro ao processar a imagem: {str(e)}"

## 6. Função para Coletar Dados do Twitter/X 

Esta função simula a integração com o Twitter/X, já que não temos acesso à API (plano gratuito).

In [ ]:
# Função para coletar dados do Twitter/X (simulada devido ao plano gratuito)
def coletar_dados_twitter(username):
    following = ["FURIA", "LOUD", "TeamLiquid"]
    interacoes = ["Assisti ao Major CS:GO 2024! #FURIA", "Jogando Valorant hoje! #esports"]
    return {"following": following, "interacoes": interacoes}

## 7. Dados Fictícios de Experiências 

Aqui criamos uma lista de experiências (eventos, produtos, workshops) que serão recomendadas com base no perfil do usuário.

In [ ]:
experiencias = pd.DataFrame({
    'EXPERIENCIA': [
        'PGL Astana 2025',
        'IEM Dallas 2025',
        'BLAST Austin Major 2025',
        'Moletom Oversized Furia x Zor Chumbo',
        'Camiseta FURIA x Adidas',
        'Jersey FURIA 2025',
        'Workshop Valorant',
        'Conteúdo Exclusivo FURIA - Bastidores CS:GO'
    ],
    'TIPO': [
        'Evento',
        'Evento',
        'Evento',
        'Produto',
        'Produto',
        'Produto',
        'Workshop',
        'Conteúdo'
    ],
    'JOGO': [
        'CS:GO',
        'CS:GO',
        'CS:GO',
        'Geral',
        'Geral',
        'Geral',
        'Valorant',
        'CS:GO'
    ],
    'TIME': [
        'FURIA',
        'FURIA',
        'FURIA',
        'FURIA',
        'FURIA',
        'FURIA',
        'Nenhum',
        'FURIA'
    ],
    'FEATURE1': [1, 1, 1, 0, 0, 0, 0, 0],
    'FEATURE2': [0, 0, 0, 1, 1, 1, 0, 0],
    'FEATURE3': [0, 0, 0, 0, 0, 0, 1, 1]
})

## 8. Função para Recomendar Experiências 

Esta função usa um algoritmo de aprendizado de máquina (KNN) para recomendar experiências personalizadas com base nos dados do usuário.

In [ ]:
def recomendar_experiencias(dados_usuario):
    if dados_usuario is None or dados_usuario.empty:
        return pd.DataFrame()

    usuario_features = [0.0, 0.0, 0.0]
    jogos_favoritos = dados_usuario['JOGOS'].iloc[0].split(',') if 'JOGOS' in dados_usuario and dados_usuario['JOGOS'].iloc[0] else []
    jogadores_favoritos = dados_usuario['JOGADORES'].iloc[0].split(',') if 'JOGADORES' in dados_usuario and dados_usuario['JOGOS'].iloc[0] else []

    if 'EVENTOS' in dados_usuario and dados_usuario['EVENTOS'].iloc[0]:
        usuario_features[0] = 1.0
        usuario_features[0] += len(dados_usuario['EVENTOS'].iloc[0].split()) * 0.1
    if 'COMPRAS' in dados_usuario and dados_usuario['COMPRAS'].iloc[0]:
        usuario_features[1] = 1.0
        usuario_features[1] += len(dados_usuario['COMPRAS'].iloc[0].split()) * 0.1
    if 'Valorant' in jogos_favoritos:
        usuario_features[2] = 1.0
    if 'CS:GO' in jogos_favoritos:
        usuario_features[0] += 0.5
    if 'League of Legends' in jogos_favoritos:
        usuario_features[2] += 0.5

    if jogadores_favoritos:
        usuario_features[0] += len(jogadores_favoritos) * 0.1
        usuario_features[1] += len(jogadores_favoritos) * 0.1

    if sum(usuario_features) == 0:
        usuario_features = [0.5, 0.5, 0.5]

    experiencias_filtradas = experiencias.copy()
    if jogos_favoritos:
        mask = experiencias_filtradas['JOGO'].apply(lambda x: x in jogos_favoritos or x == 'Geral')
        experiencias_filtradas = experiencias_filtradas[mask]
        if len(experiencias_filtradas) < 4:
            experiencias_filtradas = pd.concat([experiencias_filtradas, experiencias[~experiencias.index.isin(experiencias_filtradas.index)]], ignore_index=True)

    if experiencias_filtradas.empty:
        experiencias_filtradas = experiencias

    X = experiencias_filtradas[['FEATURE1', 'FEATURE2', 'FEATURE3']].values
    if len(X) == 0:
        return pd.DataFrame()

    knn = NearestNeighbors(n_neighbors=min(4, len(X)), algorithm='brute', metric='euclidean')
    knn.fit(X)

    distances, indices = knn.kneighbors([usuario_features])
    recomendacoes = experiencias_filtradas.iloc[indices[0]][['EXPERIENCIA', 'TIPO', 'JOGO', 'TIME']]

    tipos_desejados = ['Evento', 'Produto', 'Workshop' if 'Workshop' in experiencias_filtradas['TIPO'].values else 'Conteúdo']
    recomendacoes_final = pd.DataFrame(columns=['EXPERIENCIA', 'TIPO', 'JOGO', 'TIME'])
    tipos_presentes = []

    for i in range(len(recomendacoes)):
        rec = recomendacoes.iloc[i:i+1]
        tipo = rec['TIPO'].iloc[0]
        if tipo not in tipos_presentes:
            recomendacoes_final = pd.concat([recomendacoes_final, rec], ignore_index=True)
            tipos_presentes.append(tipo)
        if len(recomendacoes_final) >= 3:
            break

    if len(recomendacoes_final) < 3:
        for tipo in tipos_desejados:
            if tipo not in tipos_presentes:
                opcoes = experiencias_filtradas[experiencias_filtradas['TIPO'] == tipo]
                if not opcoes.empty:
                    X_opcoes = opcoes[['FEATURE1', 'FEATURE2', 'FEATURE3']].values
                    if len(X_opcoes) > 0:
                        _, idx = knn.kneighbors([usuario_features], n_neighbors=len(X_opcoes))
                        for i in idx[0]:
                            rec = opcoes.iloc[i:i+1]
                            if rec['EXPERIENCIA'].iloc[0] not in recomendacoes_final['EXPERIENCIA'].values:
                                recomendacoes_final = pd.concat([recomendacoes_final, rec], ignore_index=True)
                                tipos_presentes.append(tipo)
                                break
            if len(recomendacoes_final) >= 3:
                break

    if len(recomendacoes_final) < 3:
        for i in range(len(recomendacoes)):
            rec = recomendacoes.iloc[i:i+1]
            if rec['EXPERIENCIA'].iloc[0] not in recomendacoes_final['EXPERIENCIA'].values:
                recomendacoes_final = pd.concat([recomendacoes_final, rec], ignore_index=True)
            if len(recomendacoes_final) >= 3:
                break

    return recomendacoes_final.head(3)

## 9. Função para Sugerir Produtos ou Redes Sociais 

Esta função sugere produtos ou links de redes sociais com base nos jogadores favoritos do usuário.

In [ ]:
def sugerir_produtos_ou_redes(jogadores_selecionados):
    if not jogadores_selecionados:
        return "Nenhum jogador selecionado para sugestões."
    redes_sociais = {
        "FalleN": "Twitter: @FalleNCS",
        "yuurih": "Twitter: @yuurih",
        "KSCERATO": "Twitter: @kscerato",
        "chelo": "Twitter: @chelok1ng",
        "skullz": "Twitter: @skullzcs",
        "guerri": "Twitter: @guerri"
    }
    sugestoes = []
    for jogador in jogadores_selecionados:
        if len(sugestoes) % 2 == 0:
            sugestoes.append(f"Que tal um produto personalizado do {jogador}? Disponível na loja oficial da FURIA!")
        else:
            sugestoes.append(f"Você é fã do {jogador}? Então segue ele nas redes sociais: {redes_sociais.get(jogador, 'Não disponível')}")
    return "\n".join(sugestoes)

## 10. Interface Principal 

Esta é a interface principal do app, que junta todas as funcionalidades: formulário, upload de documentos, integração com Twitter/X e recomendações.

In [ ]:
st.title("Know Your Fan - Personalize sua Experiência com a FURIA")
dados_usuario, jogadores_selecionados = coletar_dados_usuario()

st.header("Upload de Documento")
uploaded_file = st.file_uploader("Faça upload do seu documento (ex.: RG, CPF)", type=["jpg", "png"])
if uploaded_file:
    with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
        temp_file.write(uploaded_file.getbuffer())
        temp_path = temp_file.name
    valido, mensagem = validar_documento(temp_path)
    if valido:
        st.success("Documento válido! Texto extraído: " + mensagem)
    else:
        st.error(mensagem)
    os.remove(temp_path)

st.header("Vincular Twitter/X")
twitter_username = st.text_input("Nome de usuário no Twitter/X (sem @)")
if st.button("Vincular"):
    dados_twitter = coletar_dados_twitter(twitter_username)
    if "error" not in dados_twitter:
        st.write("Páginas seguidas:", dados_twitter["following"])
        st.write("Interações relevantes:", dados_twitter["interacoes"])
        user_id = str(uuid4())
        conn = sqlite3.connect('fan_data.db')
        pd.DataFrame({
            'ID': [user_id],
            'TWITTER_USERNAME': [twitter_username],
            'FOLLOWING': [','.join(dados_twitter["following"])],
            'INTERACOES': [','.join(dados_twitter["interacoes"])]
        }).to_sql('twitter_data', conn, if_exists='append', index=False)
        conn.close()
        st.success("Dados do Twitter/X salvos! (Dados simulados para demonstração)")
    else:
        st.error(dados_twitter["error"])

if st.session_state['dados_usuario'] is not None and not st.session_state['dados_usuario'].empty:
    st.header("🎯 Recomendações Personalizadas")
    st.write("Usamos um modelo de aprendizado de máquina (K-Nearest Neighbors) para recomendar experiências com base nas suas preferências. Veja o que encontramos para você:")
    recomendacoes = recomendar_experiencias(st.session_state['dados_usuario'])
    
    if not recomendacoes.empty:
        st.divider()
        for idx, row in recomendacoes.iterrows():
            with st.container(border=True):
                st.subheader(f"⭐ {row['EXPERIENCIA']}")
                col1, col2 = st.columns(2)
                with col1:
                    st.markdown(f"**Tipo:** {row['TIPO']}")
                    st.markdown(f"**Jogo:** {row['JOGO']}")
                with col2:
                    st.markdown(f"**Time:** {row['TIME']}")
        st.divider()
    else:
        st.info("Preencha os dados para receber recomendações.")
    
    st.header("🛒 Sugestões para Você")
    sugestoes = sugerir_produtos_ou_redes(st.session_state['jogadores_selecionados'])
    st.write(sugestoes)

## 11. Como Executar o App 🚀

Para executar o app, salve este código em um arquivo `app.py` e rode o comando abaixo no terminal:

```bash
streamlit run app.py

## ⚠️ Notas importantes

- **Validação de documentos** 🖼️: Usei o Tesseract OCR localmente para evitar depender de APIs externas, como o Google Cloud Vision.
- **Integração com Twitter/X** 📱: Como estamos no plano gratuito, os dados do Twitter/X são simulados. Em um ambiente real, usaríamos a API do Twitter/X para coletar dados reais.
- **Modelo de IA**: Como mencionado, o modelo KNN foi implementado, mas as recomendações atuais dependem mais de lógica fixa do que do aprendizado de máquina.

---

### Observações Finais
- **Ajuste para o Heroku**: Note que o código acima ainda usa o caminho do Tesseract fixo para Windows (`C:\Program Files\Tesseract-OCR\tesseract.exe`). Se você planeja usar este notebook apenas para documentação e executar o app via `app.py` (onde já ajustamos o caminho do Tesseract para o Heroku), isso não será um problema. Caso queira que o notebook também seja compatível com o Heroku, você pode modificar a seção de configuração do Tesseract no Trecho 2 para:
  ```python
  # Configurar o caminho do Tesseract
  if os.name == 'nt':  # Windows
      pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
  else:  # Heroku/Linux
      pytesseract.pytesseract.tesseract_cmd = 'tesseract'  # Tesseract estará no PATH no Heroku